# 서버만들기(Flask)

In [11]:
import cv2
import numpy as np
import subprocess as sp
from matplotlib import pyplot as plt

In [12]:
def chromakey(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    patch = hsv[0:20, 0:20, :]
    minH = np.min(patch[:,:,0]) * 0.9
    maxH = np.max(patch[:,:,0]) * 1.1
    minS = np.min(patch[:,:,1]) * 0.9
    maxS = np.max(patch[:,:,1]) * 1.1
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    dest = img.copy()
    for r in range(img.shape[0]):
         for c in range(img.shape[1]) :
            if h[r,c] >= minH and h[r,c] <= maxH and   s[r,c] >= minS  and s[r,c] <= maxS : 
                dest[r,c, :] = [0,0,0]         
            else :
                dest[r,c, :] =  img[r,c,:]
    return dest            

In [2]:
%%writefile server.py
# 서버 만들기
# 장고랑 비슷한 느낌
# localhost:8000 가면 hello~ 보임

from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def index():
    return "안녕하세요^^"

@app.route('/upload')
def upload():
    return "upload~~~!!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py


In [3]:
sever = sp.Popen('python server.py',shell = True)

In [5]:
%%writefile server.py
# 서버 만들기
# 장고랑 비슷한 느낌
# localhost:8000 가면 hello~ 보임
# server.py라는 데에서 만드는 것


import cv2
import numpy as np
from flask import Flask, request

app = Flask(__name__)

@app.route('/')
def index():
    html = """
        <form action=/upload method=post enctype="multipart/form-data">
            <input type=file name=file1><input type=submit value="전송">
        </form>
    """
    return html

@app.route('/upload', methods=["post"])
def upload():
    f = request.files['file1']
    filename="./static/" + f.filename # static이라는 폴더에 저장할 것이다.
    f.save(filename)
    
    img = cv2.imread(filename)
    img = cv2.resize(img, dsize=(320,240))
    cv2.imwrite(filename, img)
  
    return "<img src=/static/" + f.filename + ">"



if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py


In [21]:
%%writefile server.py
# 서버 만들기
# 장고랑 비슷한 느낌
# localhost:8000 가면 hello~ 보임
# server.py라는 데에서 만드는 것


import cv2
import numpy as np
from flask import Flask, request
import datetime

app = Flask(__name__)

def chromakey(img, background):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    patch = hsv[0:20, 0:20, :]
    minH = np.min(patch[:,:,0]) * 0.9
    maxH = np.max(patch[:,:,0]) * 1.1
    minS = np.min(patch[:,:,1]) * 0.9
    maxS = np.max(patch[:,:,1]) * 1.1
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    dest = img.copy()
    for r in range(img.shape[0]):
         for c in range(img.shape[1]) :
            if h[r,c] >= minH and h[r,c] <= maxH and   s[r,c] >= minS  and s[r,c] <= maxS : 
                dest[r,c, :] = background[r,c,:]         
            else :
                dest[r,c, :] =  img[r,c,:]
    return dest

@app.route('/')
def index():
    html = """
        <form action=/upload method=post enctype="multipart/form-data">
            <input type="file" name='file1' accept="images/*" capture="camera">
            <input type=file name=file1><input type=submit value="전송">
        </form>
    """
    return html

@app.route('/upload', methods=["post"])
def upload():
    f = request.files['file1']
    filename="./static/" + f.filename # static이라는 폴더에 저장할 것이다.
    f.save(filename)
    

    img = cv2.imread(filename)
    img = cv2.resize(img, dsize=(320,240))
    
    img1 = cv2.imread('sun.jpg')
    img1 = cv2.resize(img1, dsize=(320,240))
  
    img = chromakey(img, img1)
    cv2.imwrite(filename, img)

    return "<img src=/static/" + f.filename + "?" + datetime.datetime.now().strftime('%H%M%S') + ">"

      
if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True, port=8000)

Overwriting server.py
